In [218]:
import numpy as np
import pandas as pd


In [544]:
#creating dataframe with all UN numbers and hazard classes

seg_df = pd.read_excel("List of UN numbers and hazard classes.xlsx")
seg_df.rename(columns={'Class': 'Hazard Class'}, inplace=True)
#seg_df['Weight'] = 0
#col_list = ['UN Number', 'Hazard Class', 'Weight', 'Proper Shipping Name']
#seg_df = seg_df[col_list]



In [545]:
seg_df.head()

,UN Number,Hazard Class,Proper Shipping Name
0,UN 0004,1.1D,"Ammonium picrate, dry or wetted with less than..."
1,UN 0005,1.1F,"Cartridges for weapons, with bursting charge"
2,UN 0006,1.1E,"Cartridges for weapons, with bursting charge"
3,UN 0007,1.2F,"Cartridges for weapons, with bursting charge"
4,UN 0009,1.2G,"Ammunition, incendiary with or without burster..."


In [548]:
values = {'UN Number': ['UN 1160', 'UN 1166', 'UN 2900', 'UN 1728', 'UN 3077', 'UN 1330', 'UN 1572', 'UN 3107'],
          #'Hazard Class': ['3', '3', '6', '8', '9', '4', '6.1', '5.2'],
          'Weight': [300, 800, 450, 1200, 500, 3000, 1001, 1100],
          'Bulk Container?': ['N', 'N', 'Y', 'N', 'Y', 'N', 'N', 'N']}

df = pd.DataFrame.from_dict(values)
df.head(6)

,UN Number,Weight,Bulk Container?
0,UN 1160,300,N
1,UN 1166,800,N
2,UN 2900,450,Y
3,UN 1728,1200,N
4,UN 3077,500,Y
5,UN 1330,3000,N


In [550]:
#concatenate df into seg_df

new_df = seg_df.merge(df, how='outer', on='UN Number')
new_df.head()

,UN Number,Hazard Class,Proper Shipping Name,Weight,Bulk Container?
0,UN 0004,1.1D,"Ammonium picrate, dry or wetted with less than...",NaN,NaN
1,UN 0005,1.1F,"Cartridges for weapons, with bursting charge",NaN,NaN
2,UN 0006,1.1E,"Cartridges for weapons, with bursting charge",NaN,NaN
3,UN 0007,1.2F,"Cartridges for weapons, with bursting charge",NaN,NaN
4,UN 0009,1.2G,"Ammunition, incendiary with or without burster...",NaN,NaN


In [552]:
#drop all rows with weight == 0

df = new_df[new_df['Weight'].notna()]

df.head(10)

,UN Number,Hazard Class,Proper Shipping Name,Weight,Bulk Container?
502,UN 1160,3,Dimethylamine solution,300.0,N
508,UN 1166,3,Dioxolane,800.0,N
633,UN 1330,4.1,Manganese resinate,3000.0,N
821,UN 1572,6.1,Cacodylic acid,1001.0,N
953,UN 1728,8,Amyltrichlorosilane,1200.0,N
1812,UN 2900,6.2,"Infectious substance, affecting animals",450.0,Y
1924,UN 3077,9,"Environmentally hazardous substance, solid, n....",500.0,Y
1953,UN 3107,5.2,"Organic peroxide type E, liquid",1100.0,N


In [583]:
#code template for user input: https://www.reddit.com/r/learnpython/comments/rjx5kz/pandas_data_frame_using_user_input/


def placarding(df):

    df2 = df.groupby([
       'Hazard Class'], as_index=False)['Weight'].sum()
    print(df2)
    for row, col in df.iterrows():
        if col['Bulk Container?'] == 'Y':
            print(f"Bulk placard for {col['UN Number']}, {col['Hazard Class']}") 

        if col['Bulk Container?'] == 'N':
            if sum(df['Weight']) < 1000:
                print("non-bulk placarding not required")
    for row, col in df2.iterrows():
        if col['Hazard Class'] != 9:
            if col['Weight'] > 1000:
                print(f"Placard for Class {col['Hazard Class']}")
            if col['Weight'] <= 1000:
                print(f"Use Dangerous placard for Class {col['Hazard Class']} (placarding for Class {col['Hazard Class']} is also acceptable)")

    return df2

    


#def input():
#	input UN number as num
#	input weight as weight
#	input bulk yes/no
#		try:
#	 		where df[UN number] == num, insert weight and bulk
#		except:
#			print "invalid UN number"
#			pass
#	input do you have more to enter?
#		if yes:
#			run input function again
#		if no:
#			pass
#	return placarding()	
			
		

In [585]:
df2 = placarding(df)

  Hazard Class  Weight
0            3  1100.0
1          4.1  3000.0
2          5.2  1100.0
3          6.1  1001.0
4          6.2   450.0
5            8  1200.0
6            9   500.0
Bulk placard for UN 2900, 6.2
Bulk placard for UN 3077, 9
Placard for Class 3
Placard for Class 4.1
Placard for Class 5.2
Placard for Class 6.1
Use Dangerous placard for Class 6.2 (placarding for Class 6.2 is also acceptable)
Placard for Class 8


In [322]:
#build lists of incompatabilities
#we are going to ignore classes 1 and 7 because my employer does not transport those
#we will feed df2 from the function above into this function

#def segregationCheck(df2):
print(df2)
for row, col in df2.iterrows():
    if col['Weight'] > 1000 and col['Hazard Class'] == '2.1':
        if df2['Hazard Class'].isin(['2.3']).any():
            print("STOP! This shipment may be forbidden or require segregation between classes 2.1 and 2.3.")
        if df2['Hazard Class'].isin(['6.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between classes 2.1 and 6.1.")

#looks like i need to create an if statement for each class
    
    if col['Hazard Class'] == '2.3':
        if df2['Hazard Class'].isin(['3']).any():
             print("STOP! This shipment may be forbidden or require segregation between class 2.3 and class 3")
        if df2['Hazard Class'].isin(['4.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 2.3 and class 4.1")
        if df2['Hazard Class'].isin(['4.2']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 2.3 and class 4.2")
        if df2['Hazard Class'].isin(['4.3']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 2.3 and class 4.3")
        if df2['Hazard Class'].isin(['5.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 2.3 and class 5.1")
        if df2['Hazard Class'].isin(['5.2']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 2.3 and class 5.2")
        if df2['Hazard Class'].isin(['8']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 2.3 and class 8")
        
   
#duplicate class 3 = pass list into isin?    
    if col['Weight'] > 1000 and col['Hazard Class'] == '3':

        if df2['Hazard Class'].isin(['5.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 3 and class 5.1")
        if df2['Hazard Class'].isin(['6.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 3 and class 6.1")
   
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '4.1': 
        if df2['Hazard Class'].isin(['6.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 4.1 and class 6.1")
        if df2['Hazard Class'].isin(['8']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 4.1 and class 8")
   
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '4.2':    
        if df2['Hazard Class'].isin(['6.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 4.2 and class 6.1")
        if df2['Hazard Class'].isin(['8']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 4.2 and class 8")
   
   
    
    if col['Hazard Class'] == '4.3': 
        if df2['Hazard Class'].isin(['6.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 4.3 and class 6.1")
        if df2['Hazard Class'].isin(['8']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 4.3 and class 8")
    
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '5.1':  
        if df2['Hazard Class'].isin(['6.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 5.2 and class 6.1")
        if df2['Hazard Class'].isin(['8']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 5.2 and class 8")
   
    
    if col['Weight'] > 1000 and col['Hazard Class'] == '5.2':   
        if df2['Hazard Class'].isin(['6.1']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 5.2 and class 6.1")
        if df2['Hazard Class'].isin(['8']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 5.2 and class 8")
    
    if col['Hazard Class'] == '6.1': 
        if df2['Hazard Class'].isin(['8']).any():
            print("STOP! This shipment may be forbidden or require segregation between class 6.1 and class 8")
    

    


  Hazard Class  Weight
0            3    1100
1            4    3000
2          5.2    1100
3            6     450
4          6.1    1001
5            8    1200
6            9     500
STOP! This shipment may be forbidden or require segregation between class 3 and class 6.1
STOP! This shipment may be forbidden or require segregation between class 5.2 and class 6.1
STOP! This shipment may be forbidden or require segregation between class 5.2 and class 8
STOP! This shipment may be forbidden or require segregation between class 6.1 and class 8
